In [11]:
from pymongo import MongoClient
import os
from getpass4 import getpass
from datetime import datetime
import pandas as pd

In [6]:
MONGODB_URI = (
    os.environ["MONGODB_URI"]
    if "MONGODB_URI" in os.environ
    else input("MongoDB Connection String: ")
)

In [7]:
client = MongoClient(MONGODB_URI)
db = client["genai_inventory_classification"]
collection = db["orders"]


In [10]:
pipeline_abc_analysis = [
    {
        '$match': {
            'purchaseTimestamp': {
                '$lt': datetime(2018, 6, 30, 22, 0, 0)
            }
        }
    }, {
        '$unwind': {
            'path': '$items'
        }
    }, {
        '$group': {
            '_id': '$items.productId', 
            'revenue': {
                '$sum': '$items.price'
            }, 
            'category': {
                '$max': '$items.productCategoryNameEnglish'
            }
        }
    }, {
        '$setWindowFields': {
            'partitionBy': None, 
            'sortBy': {
                'revenue': -1
            }, 
            'output': {
                'cumulativeRevenue': {
                    '$sum': '$revenue', 
                    'window': {
                        'documents': [
                            'unbounded', 'current'
                        ]
                    }
                }, 
                'totalRevenue': {
                    '$sum': '$revenue'
                }
            }
        }
    }, {
        '$project': {
            'revenue': 1, 
            'category': 1, 
            'cumulativeRevenue': 1, 
            'totalRevenue': 1, 
            'cumulativePercentage': {
                '$divide': [
                    '$cumulativeRevenue', '$totalRevenue'
                ]
            }
        }
    }, {
        '$project': {
            'revenue': 1, 
            'category': 1, 
            'cumulativeRevenue': 1, 
            'totalRevenue': 1, 
            'cumulativePercentage': 1, 
            'class': {
                '$switch': {
                    'branches': [
                        {
                            'case': {
                                '$lte': [
                                    '$cumulativePercentage', 0.6
                                ]
                            }, 
                            'then': 'A'
                        }, {
                            'case': {
                                '$and': [
                                    {
                                        '$gt': [
                                            '$cumulativePercentage', 0.6
                                        ]
                                    }, {
                                        '$lte': [
                                            '$cumulativePercentage', 0.85
                                        ]
                                    }
                                ]
                            }, 
                            'then': 'B'
                        }
                    ], 
                    'default': 'C'
                }
            }
        }
    },
]

In [12]:
results_abc = collection.aggregate(pipeline_abc_analysis)
df_results = pd.DataFrame(list(results_abc))
df_results.head()

,_id,revenue,category,cumulativeRevenue,totalRevenue,cumulativePercentage,class
0,bb50f2e236e5eea0100680137654686c,50355.00,health_beauty,50355.00,11839437.79,0.004253,A
1,d6160fb7873f184099d9bc95e30376af,48899.34,computers,99254.34,11839437.79,0.008383,A
2,6cdd53843498f92890544667809f1595,47450.20,health_beauty,146704.54,11839437.79,0.012391,A
3,d1c427060a0f73f6b889a5c7c61f2ac4,44934.70,computers_accessories,191639.24,11839437.79,0.016187,A
4,99a4788cb24856965c36a24e339b6058,41726.26,bed_bath_table,233365.50,11839437.79,0.019711,A


In [16]:
pipeline_abc_verification = [
    {
        '$match': {
            'purchaseTimestamp': {
                '$lt': datetime(2018, 6, 30, 22, 0, 0)
            }
        }
    }, {
        '$unwind': {
            'path': '$items'
        }
    }, {
        '$group': {
            '_id': '$items.productId', 
            'revenue': {
                '$sum': '$items.price'
            }, 
            'category': {
                '$max': '$items.productCategoryNameEnglish'
            }
        }
    }, {
        '$setWindowFields': {
            'partitionBy': None, 
            'sortBy': {
                'revenue': -1
            }, 
            'output': {
                'cumulativeRevenue': {
                    '$sum': '$revenue', 
                    'window': {
                        'documents': [
                            'unbounded', 'current'
                        ]
                    }
                }, 
                'totalRevenue': {
                    '$sum': '$revenue'
                }
            }
        }
    }, {
        '$project': {
            'revenue': 1, 
            'category': 1, 
            'cumulativeRevenue': 1, 
            'totalRevenue': 1, 
            'cumulativePercentage': {
                '$divide': [
                    '$cumulativeRevenue', '$totalRevenue'
                ]
            }
        }
    }, {
        '$project': {
            'revenue': 1, 
            'category': 1, 
            'cumulativeRevenue': 1, 
            'totalRevenue': 1, 
            'cumulativePercentage': 1, 
            'class': {
                '$switch': {
                    'branches': [
                        {
                            'case': {
                                '$lte': [
                                    '$cumulativePercentage', 0.6
                                ]
                            }, 
                            'then': 'A'
                        }, {
                            'case': {
                                '$and': [
                                    {
                                        '$gt': [
                                            '$cumulativePercentage', 0.6
                                        ]
                                    }, {
                                        '$lte': [
                                            '$cumulativePercentage', 0.85
                                        ]
                                    }
                                ]
                            }, 
                            'then': 'B'
                        }
                    ], 
                    'default': 'C'
                }
            }
        }
    }, {
        '$setWindowFields': {
            'partitionBy': None, 
            'output': {
                'totalProducts': {
                    '$count': {}, 
                    'window': {
                        'documents': [
                            'unbounded', 'unbounded'
                        ]
                    }
                }
            }
        }
    }, {
        '$group': {
            '_id': '$class', 
            'count': {
                '$count': {}
            }, 
            'totalProducts': {
                '$first': '$totalProducts'
            }
        }
    }, {
        '$project': {
            '_id': 0, 
            'class': '$_id', 
            'percentage': {
                '$divide': [
                    '$count', '$totalProducts'
                ]
            }
        }
    }, {
        '$sort': {
            'class': 1
        }
    }
]

In [ ]:
results_abc_verification = collection.aggregate(pipeline_abc_verification)
df_results = pd.DataFrame(list(results_abc_verification))
df_results.head()

,class,percentage
0,A,0.097530
1,B,0.240318
2,C,0.662151
